In [17]:
from torchvision import models
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from PIL import Image
import csv
import os
from sklearn.metrics import classification_report


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
if torch.cuda.device_count() > 1:
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    model = torch.nn.DataParallel(model)
    

Using device: cuda:0


In [12]:
def create_csv(base_path,output_csv_file):
    with open(output_csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Label"])  
    
        for folder in os.listdir(base_path):
            folder_path = os.path.join(base_path, folder)
            
            if os.path.isdir(folder_path):
                for filename in os.listdir(folder_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                        writer.writerow([filename, folder])
                    else:
                        print(filename,folder,"CHECKING!")

train_path = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\train"
train_csv = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\image_labels.csv"
create_csv(train_path,train_csv)


In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

In [15]:
dataset_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\train' 
testing_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\test' 
train_dataset = ImageFolder(root=dataset_path, transform=transform)
test_dataset = ImageFolder(root=testing_path, transform=transform)
batch_size = 32
num_workers = 4
pin_memory = True

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)


In [16]:
#BEST ONE YET
model = models.resnext50_32x4d(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 25
for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)  
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')



C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 0.8617
Epoch 2: Train Loss: 0.4083
Epoch 3: Train Loss: 0.1732
Epoch 4: Train Loss: 0.1254
Epoch 5: Train Loss: 0.1123
Epoch 6: Train Loss: 0.0839
Epoch 7: Train Loss: 0.0607
Epoch 8: Train Loss: 0.0457
Epoch 9: Train Loss: 0.0574
Epoch 10: Train Loss: 0.0610
Epoch 11: Train Loss: 0.1007
Epoch 12: Train Loss: 0.0966
Epoch 13: Train Loss: 0.0742
Epoch 14: Train Loss: 0.0320
Epoch 15: Train Loss: 0.0692
Epoch 16: Train Loss: 0.0672
Epoch 17: Train Loss: 0.0421
Epoch 18: Train Loss: 0.0156
Epoch 19: Train Loss: 0.0764
Epoch 20: Train Loss: 0.0405
Epoch 21: Train Loss: 0.0634
Epoch 22: Train Loss: 0.0227
Epoch 23: Train Loss: 0.0572
Epoch 24: Train Loss: 0.0488
Epoch 25: Train Loss: 0.0279


In [18]:
def get_predictions(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']))

                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.60      0.49      0.54       221
Cassava Brown Streak Disease (CBSD)       0.80      0.68      0.73       431
         Cassava Green Mottle (CGM)       0.81      0.72      0.76       493
       Cassava Mosaic Disease (CMD)       0.84      0.83      0.83       528
                            Healthy       0.60      0.81      0.69       466

                           accuracy                           0.74      2139
                          macro avg       0.73      0.71      0.71      2139
                       weighted avg       0.75      0.74      0.74      2139



In [19]:
import torch
torch.save(model.state_dict(), 'final.pth')